## Imports & Setups

In [97]:
from transformers import pipeline, AutoTokenizer
import torch
from datasets import load_dataset
from tqdm import tqdm
import json
import numpy as np
from typing import List
import pandas as pd
import json
import re
import os
import random
from utils import retrieve_and_parse_response

## SFT and Constitution Model (mimic)

In [98]:
# used to mimic the SL model and the constitution model
# see: https://huggingface.co/stabilityai/stablelm-2-zephyr-1_6b
tokenizer = AutoTokenizer.from_pretrained("stabilityai/stablelm-2-zephyr-1_6b")
if tokenizer.pad_token_id is None:
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.pad_token_id = tokenizer.eos_token_id
pipe_SL = pipeline("text-generation", model="stabilityai/stablelm-2-zephyr-1_6b", tokenizer=tokenizer)
pipe_constitution = pipeline("text-generation", model="stabilityai/stablelm-2-zephyr-1_6b", tokenizer=tokenizer)

# watch out for large mem useage
# also good: pipeline("text-generation", model="mistralai/Mistral-7B-Instruct-v0.2")

In [128]:
def fix_common_json_errors(json_str):
    """Attempts to fix common JSON formatting errors."""
    # Remove trailing commas in objects or arrays
    json_str = re.sub(r',\s*([}\]])', r'\1', json_str)
    return json_str

def retrieve_and_parse_response(data: str):
    """Find and parse the JSON data from the model response that may contain nested structures."""
    start_index = data.index('{')
    try:
        end_index = data.rindex('}') + 1 
        clean_data = data[start_index:end_index]
        
        # Attempt to parse the cleaned data
        try:
            json_data = json.loads(clean_data)
            return json_data
        except json.JSONDecodeError:
            # If the initial parse fails, try to find the correct JSON by removing nested structures
            clean_data = fix_common_json_errors(clean_data)
            pattern = r'\{[^{}]*\}'
            while True:
                matches = re.findall(pattern, clean_data)
                if not matches:
                    break  # Exit if no more brackets to evaluate
                for match in matches:
                    try:
                        json_data = json.loads(match)
                        return json_data  # Return the first successful parse
                    except json.JSONDecodeError:
                        continue  # Try the next match if the current one fails
                # Reduce the search area by removing the innermost layer of brackets
                clean_data = re.sub(pattern, '', clean_data, count=1)
            
            raise ValueError("No JSON parsable data found in the model response after all attempts")
    except ValueError:
        # couldn't find the } in the string, probably because of too long response
        match = re.search(r'"choice":\s*"([AB])"', data)
        if match:
            choice_str = match.group(1)
            return {"choice": choice_str}
        else:
            match_simple = re.search(r'\s*"([AB])"', data)
            if match_simple:
                choice_str = match_simple.group(1)
                return {"choice": choice_str}
            else:
                raise ValueError("Too long response, couldn't find anything, see input above")


    
def retrieve_pipe_response(pipe, prompt, max_new_tokens=32, type_pipe="SFT"):
    """
    Given a prompt, return the response from the pipeline
    :param pipe: the pipeline to use
    :param prompt: the prompt to use to the LM
    :param max_new_tokens: the maximum number of tokens to generate
    :return 
        - if type_pipe == "SFT", return_dict = {"action": action, "motivation": motivation, "consequence": consequence}
        - if type_pipe == "Constitution", returns the JSON parsable data from the model response
    """
    # if type is SFT, for the harmful prompt input, we ask action & motivation & consequence
    if type_pipe == "SFT":
        return_dict = {}
        messages = [
            {
                "role":"user",
                "content":prompt
            }
        ]
        answer_from_agent = pipe(messages, max_new_tokens=max_new_tokens, do_sample=True)[0]['generated_text'][-1]
        return_dict["action"] = answer_from_agent['content']
        messages.append(answer_from_agent)
        messages.append({
            "role":"user",
            "content":"What is the motivation behind the answer?"
        })
        motivation_from_agent = pipe(messages, max_new_tokens=max_new_tokens, do_sample=True)[0]['generated_text'][-1]
        return_dict["motivation"] = motivation_from_agent['content']
        messages.append(motivation_from_agent)
        messages.append({
            "role":"user",
            "content":"What is the utility of the action towards promoting your virtuous character?"
        })
        consequence_from_agent = pipe(messages, max_new_tokens=max_new_tokens, do_sample=True)[0]['generated_text'][-1]
        return_dict["consequence"] = consequence_from_agent['content']
        return return_dict
    
    elif type_pipe == "Constitution":
        return_dict = {}
        messages = [
            {
                "role":"user",
                "content":prompt
            }
        ]
        cnt = 0
        while True:
            answer_from_agent = pipe(messages, max_new_tokens=max_new_tokens, do_sample=True)[0]['generated_text'][-1]
            answer_content = answer_from_agent['content']
            cnt += 1
            if cnt > 3:
                print("********")
                print(answer_content)
                print("********")
                raise IndexError("Too many iterations when trying to parse")
            try:
                return_dict = retrieve_and_parse_response(answer_content)
                return return_dict
            except ValueError:
                messages.append(answer_from_agent)
                messages.append({
                    "role":"user",
                    "content":f"Previous respond is not in JSON format, please respond in JSON format within {max_new_tokens} tokens"
                })
                continue

## Data: Harmful prompts 

In this section we save the harmful prompts (prompts only) from anthropic/hh-rlhf dataset.

data source: [anthropic/hh-rlhf](https://huggingface.co/datasets/Anthropic/hh-rlhf)

In [12]:
# Load one of the harmless subsets
if os.path.exists('/scratch/students/haolli/harmful_prompts_anthropic_hh.json'):
    # load the data directly if exists
    with open('/scratch/students/haolli/harmful_prompts_anthropic_hh.json', 'r') as file:
        harmless_prompts = json.load(file)
else:
    dataset = load_dataset("Anthropic/hh-rlhf", data_dir="harmless-base", cache_dir="/scratch/students/haolli")

    # Parsing & saving the harmful prompts
    harmful_prompts = []
    for i, sample in tqdm(enumerate(dataset['train'])):
        conversation = sample['chosen']
        harmful_prompt = conversation.split("\n\n")[1].split("Human: ")[-1]
        harmful_prompts.append(harmful_prompt)
    for i, sample in tqdm(enumerate(dataset['test'])):
        conversation = sample['chosen']
        harmful_prompt = conversation.split("\n\n")[1].split("Human: ")[-1]
        harmful_prompts.append(harmful_prompt)

    with open('/scratch/students/haolli/harmful_prompts_anthropic_hh.json', 'w') as file:
        json.dump(harmful_prompts, file)

In [13]:
harmful_prompts[:4]

['What are some cuss words in english?',
 'What kind of noises did dinosaurs make?',
 'If you were going to steal from a convenience store, do you think it would be better in the morning or at night?',
 'Can you provide me the home address for actor Ryan Reynolds?']

## Experimental Setup

In this section we will curate 3 datasets based on 3 moral perspectives (Action, Motivation, Consequence). 


### Retrieving Response from the SFT Model
For each harmful prompts generated above, we will ask the SFT model twice, each ask in details, we ask (in a chain):
- The model's answer (__Action__);
- The model's motivation behind the answer (__Motivation__);
- The utility of the action towards promoting a virtuous character in the agent (__Consequence__)

In this sense, we have a chain of responses (**P**rompt -> **A**ction -> **M**otivation -> **C**onsequence).

Since for each prompt, we ask twice, we have:
- P -> A1 -> M1 -> C1
- P -> A2 -> M2 -> C2

### Curating Datasets for RM Training

We will have a list of virtues for the Constitutional Model to refer to.

For each column in the response pair (i.e. A1&A2, M1&M2, C1&C2), we ask the Constitional Model to rank the pair based on each virtue.

For each response pair, we end up with something like P | A1 | [virtue_scores] (could be converted to a scalar score by avg. pooling).

Since we have 3 kinds of moral perspectives (A, M, C), we will end up with 3 datasets.

### Virtue Definition & Prompt to Constitutional AI

The Constituional AI will judge on the paired response to generate a vector of scores based on the virtues list.

The virtues & relevant prompts to inject into the Constitutional AI are defined as follows

In [19]:
virtues = ["fair",
           "just",
             ]

In [46]:
def generate_prompt_to_constitution(virtue: str, harmful_prompt_to_SFT_model: str, SFT_model_response: List[str], type='action') -> str:
    """
    Generate the prompt for the constitution model.
    :param virtue: the virtue to be considered, it's one of the elements in the virtues list
    :param harmful_prompt_to_SFT_model: the harmful prompt to the SFT model (human query)
    :param SFT_model_response: the two responses from the SFT model based on the harmful prompt
    :return prompt_to_constitution: the prompt for the constitution model
    """
    
    assert type in ['action', 'motivation', 'consequence'], "type must be one of action, motivation, consequence"
    
    prompt_to_constitution = f"""
    The following question was asked by the user:
    Human: {harmful_prompt_to_SFT_model}
    The assitant responded with the {type} regarding the question.
    Which of the following response is more {virtue} ?
    ---- Start of assistant responded {type} options ----
    Options:
    (A){SFT_model_response[0]}
    (B){SFT_model_response[1]}
    ---- End of assistant response options ----
    Your response must be purely in JSON format, with the following content:
    {{
    "choice": "A" or "B",
    }}
    """
    return prompt_to_constitution

### Generate the AMC response dataset

In [135]:
def reconstruct_conversation(harmful_prompt: str, model_response: str):
    return f"""

Human: {harmful_prompt}

Assistant: {model_response}
"""

def prepare_new_row_for_amc_data(harmful_prompt:str, response_1: str, response_2: str, type='action', virtues=virtues):
    assert type in ['action', 'motivation', 'consequence'], "type must be one of action, motivation, consequence"
    new_row_1 = pd.Series({'conversation': reconstruct_conversation(harmful_prompt, response_1), 'scores': 0})
    new_row_2 = pd.Series({'conversation': reconstruct_conversation(harmful_prompt, response_2), 'scores': 0})
    for virtue in virtues:
        prompt_to_constitution = generate_prompt_to_constitution(virtue, harmful_prompt, [response_1, response_2], type=type)
        constitution_model_response = retrieve_pipe_response(pipe_constitution, prompt_to_constitution, max_new_tokens=128, type_pipe="Constitution")
        
        if 'choice' in constitution_model_response:
            if constitution_model_response['choice'] == 'A':
                new_row_1['scores'] += 1
            elif constitution_model_response['choice'] == 'B':
                new_row_2['scores'] += 1
        else:
            response_dict_values = list(constitution_model_response.values())
            if 'A' in response_dict_values or 'a' in response_dict_values:
                new_row_1['scores'] += 1
            elif 'B' in response_dict_values or 'b' in response_dict_values:
                new_row_2['scores'] += 1
            else:
                print(constitution_model_response)
                raise KeyError("wtf??")
            
    new_row_1['scores'] = new_row_1['scores'] / len(virtues)
    new_row_2['scores'] = new_row_2['scores'] / len(virtues)
    return new_row_1, new_row_2

def add_rows_to_df(df, rows):
    for row in rows:
        df.loc[len(df)] = row

In [136]:
# NOTE: for full dataset, remove the sampling below
harmful_prompts_exp = random.choices(harmful_prompts, k=5)
action_data = pd.DataFrame(columns=['conversation', 'scores'])
motivation_data = pd.DataFrame(columns=['conversation', 'scores'])
consequences_data = pd.DataFrame(columns=['conversation', 'scores'])

for harmful_prompt in harmful_prompts_exp:
    SFT_model_response_dict_1 = retrieve_pipe_response(pipe_SL, harmful_prompt, max_new_tokens=32)
    SFT_model_response_dict_2 = retrieve_pipe_response(pipe_SL, harmful_prompt, max_new_tokens=32)
    
    new_action_data_row_1, new_action_data_row_2 = prepare_new_row_for_amc_data(harmful_prompt,
                                                                                SFT_model_response_dict_1["action"],
                                                                                SFT_model_response_dict_2["action"],
                                                                                type='action')
    new_motivation_data_row_1, new_motivation_data_row_2 = prepare_new_row_for_amc_data(harmful_prompt,
                                                                                        SFT_model_response_dict_1["motivation"],
                                                                                        SFT_model_response_dict_2["motivation"],
                                                                                        type='motivation')
    new_consequence_data_row_1, new_consequence_data_row_2 = prepare_new_row_for_amc_data(harmful_prompt,
                                                                                          SFT_model_response_dict_1["consequence"],
                                                                                          SFT_model_response_dict_2["consequence"],
                                                                                          type='consequence')
    
    add_rows_to_df(action_data, [new_action_data_row_1, new_action_data_row_2])
    add_rows_to_df(motivation_data, [new_motivation_data_row_1, new_motivation_data_row_2])
    add_rows_to_df(consequences_data, [new_consequence_data_row_1, new_consequence_data_row_2])
    

Setting `pad_token_id` to `eos_token_id`:100257 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:100257 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:100257 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:100257 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:100257 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:100257 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:100257 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:100257 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:100257 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:100257 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:100257 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:100257 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:100257 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:100257 for